In [1]:
# export
from dotenv import load_dotenv
load_dotenv()

import os
import pickle
from datetime import datetime


import pandas as pd
import numpy as np
from datetime import datetime
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
from datetime import datetime
import btalib

#custom library
import MLTrain as mlt

pd.set_option('display.max_rows', 500)


import logging

# Gets or creates a logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

In [44]:
# export
def f_get_coinlist():
    coin_list_df = pd.read_csv("crypto_lu.csv")
    coin_list_df["Symbolusd"] =  coin_list_df["Symbol"].apply(lambda x: x + "USDT" )
    coin_list_df["Keywords"]  = coin_list_df.apply(lambda row: row["Name"].lower() + "," + 
                                   row["Symbol"].lower() + "," + 
                                   row["Symbolusd"].lower() + "," + 
                                   row["Keywords"].lower()  , axis = 1)


    coin_list_df = coin_list_df.head(25)

    coin_list = list(coin_list_df.Symbolusd)
    return coin_list


def f_get_binance_client():
    api_key = os.environ.get('BINANCE_API')
    api_secret = os.environ.get('BINANCE_SECRET')
    return Client(api_key, api_secret)


def f_load_allcoins(coin_list = None):
    
    client = f_get_binance_client()
    coin_list = f_get_coinlist()
    dict_all = {}
    for coin in coin_list:  
        dict_coin = {}    
        logger.info('Initial load {x}'.format(x=coin))
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_2HOUR, "365 day ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('Initial load {x} FAILED'.format(x=coin))
            continue
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



# fetch 1 minute klines for the last day up until now
def f_update_lastNdays(dict_all=None):
    
    if dict_all is None:
        file = open("dict_all.pkl","rb")
        dict_all = pickle.load(file)
    
    client = f_get_binance_client()
    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_2HOUR, "2 day ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})
        
    pickle_out = open("dict_all.pkl","wb")
    pickle.dump(dict_all, pickle_out)
    pickle_out.close()

    return dict_all



# fetch 1 minute klines for the last day up until now
def f_update_lastprice(dict_all=None):
    
    client = f_get_binance_client()
    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1MINUTE, "1 minute ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



def f_get_all_price(dict_all_coins):
    #dict_all_coins = f_load_allcoins(coin_list = None)
    #dict_all_coins = f_update_lastprice(dict_all=dict_all_coins)
    
    df_all_coins = pd.DataFrame(columns= ["coin", "dt", "price", "vol", "tradefreq"])
    for coin in dict_all_coins.keys():
        dict_coin = dict_all_coins.get(coin)
        listx = []
        for dt in dict_coin.keys():
            listx.append([coin, dt, dict_coin.get(dt)[1], dict_coin.get(dt)[2], dict_coin.get(dt)[3]])
        df_all_coins = df_all_coins.append(pd.DataFrame(data = listx,
                                         columns= ["coin", "dt", "price", "vol", "tradefreq"]
                                        )
                           )
    df_all_coins.dt = df_all_coins.dt.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')  )
    df_all_coins['last_time'] = 0
    for coin in df_all_coins.coin.unique():
        max_dt = np.max(df_all_coins.loc[df_all_coins.coin == coin, 'dt'])
        df_all_coins.loc[(df_all_coins.coin == coin) & (df_all_coins.dt == max_dt ) , 'last_time'] = 1

    df_all_coins.set_index("dt", drop=True, inplace = True )
    df_all_coins.sort_index(inplace=True)
    df_all_coins["is_day_end"] = 0
    df_all_coins.loc[[True if x.strftime("%H:%M:%S") == '00:00:00' else False for x in list(df_all_coins.index)], "is_day_end"] = 1  
    df_all_coins.price = df_all_coins.price.astype(float)
    df_all_coins.tradefreq= df_all_coins.tradefreq.astype(float)
    df_all_coins.vol= df_all_coins.vol.astype(float)
    
    
    return df_all_coins
    


def f_calc_indicators(df_all_coins):

    #df_all_coins = f_get_all_price()
    
    for coin in df_all_coins.coin.unique():
        index_t = (df_all_coins.coin == coin)
        #df_all_coins.loc[index_t, 'ma12'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=12).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma24'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=24).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma48'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma148'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi14'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=14).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi28'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=28).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi42'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=42).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi48'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi72'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=72).df.values.tolist()
        df_all_coins.loc[index_t, ["macd12", "signal12", "hist12"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=12, pslow=24, psignal=9).df.values.tolist()
        df_all_coins.loc[index_t, ["macd6", "signal6", "hist6"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=6, pslow=12, psignal=4).df.values.tolist()

    df_all_coins['macd12_ratio'] = df_all_coins['macd12'] / df_all_coins['price']
    df_all_coins['signal12_ratio'] = df_all_coins['signal12'] / df_all_coins['price']
    
    
    return df_all_coins


def f_get_last_minute_rows(v_df, rnk = 1):
    #df = f_calc_indicators()
    df = v_df.copy()
    df.reset_index(inplace=True)
    df.loc[:, "rnk_dt"] =  df.groupby("coin")["dt"].rank("dense", ascending=False)
    df = df.loc[df.rnk_dt == rnk, :]
    df.set_index("dt", inplace = True)
    df.drop(['rnk_dt'], axis=1, inplace = True)
    return df
    
    

    
    

In [3]:
# export
def set_target(row, margin):
    meanx = np.nanmean([row["next1price"], row["next2price"], row["next3price"] ])
    if meanx > row["price"] * (1.00 + margin):
        return 1
    elif meanx < row["price"] * (1.00 - margin):
        return -1
    else:
        return 0

    
def set_target_numeric(row):
    return np.nanmean([row["next1price"], row["next2price"], row["next3price"] ])    
    
def f_transform_df_target(df_all_coins, shift_period = 2):    
    Target_Type = 'Binary'
    df_all_coins["next1price"] = df_all_coins.groupby(['coin'])['price'].shift(-1*shift_period)
    df_all_coins["next2price"] = df_all_coins.groupby(['coin'])['price'].shift(-2*shift_period)
    df_all_coins["next3price"] = df_all_coins.groupby(['coin'])['price'].shift(-4*shift_period)

    df_all_coins["Target"] = 0
       

    
    if Target_Type == 'Numeric':    
        #Numeric Target
        df_all_coins.loc[df_all_coins.last_time == 0, "Target"] = \
                df_all_coins.loc[df_all_coins.last_time == 0, :].apply(lambda row: set_target_numeric(row), axis = 1)    
    else:
        #Binary Target
        df_all_coins.loc[df_all_coins.last_time == 0, "Target"] = \
                df_all_coins.loc[df_all_coins.last_time == 0, :].apply(lambda row: set_target(row, 0.02), axis = 1)
    
    pickle_out = open("df_all.pkl","wb")
    pickle.dump(df_all_coins, pickle_out)
    pickle_out.close()
    
    return df_all_coins


def f_prep_df_to_ML(df):
    df_tmp = df.copy()
    df_tmp.dropna(inplace = True)
    y = df_tmp.loc[:, ["Target"]]
    X = df_tmp.loc[:, [ x for x in df_tmp.columns if x not in (["Target" 
                                                                ,"coin" 
                                                                ,"next1price" 
                                                                ,"next2price" 
                                                                ,"next3price" 
                                                                ,"price"
                                                               # ,"is_day_end"
                                                               # ,"last_time"
                                                               ])  ]]

    return X, y 

In [45]:
dict_init = f_load_allcoins()

APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.


In [46]:
dict_tmp = f_update_lastNdays()
dict_tmp = f_update_lastprice(dict_tmp)
df = f_get_all_price(dict_tmp)

In [5]:
df = f_calc_indicators(df)
df = f_transform_df_target(df)

/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
df1 = f_get_last_minute_rows(df, rnk = 9)

df1.loc[df1.coin == 'DOGEUSDT', :]

X = mlt.f_prep_df_to_ML(df1.loc[df1.coin == 'DOGEUSDT', :], is_train = False)[0]

X

,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio
dt,,,,,,,,,,,,,,,,,
2021-05-07 16:00:00,1.620729e+09,929333.0,0,0,62.434687,64.03126,63.831153,63.637488,62.940641,0.036308,0.043398,-0.007089,0.010032,0.009517,0.000515,0.057198,0.068365


In [15]:
X

,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio
dt,,,,,,,,,,,,,,,,,
2021-05-08 16:00:00,3.209402e+09,1697858.0,0,0,60.642484,62.989398,63.200064,63.12382,62.694502,0.041253,0.043367,-0.002114,0.017587,0.020974,-0.003387,0.061049,0.064177


In [42]:
file = open("model_dict.pkl","rb")
model_dict = pickle.load(file)
model_dict.get("DOGEUSDT")[0].predict_proba(X)

array([[0.26925945, 0.02323879, 0.70750175]])

In [ ]:
from exporter import export
export("Financial_Indicators.ipynb","Financial_Indicators.py")

In [ ]:
# export
if __name__ == "__main__":
    print("main")